In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import checker
import utils

# Proszę nie zmieniać seedu, to nam zapewni, ze bedzięmy pracować na tych samych datasetach
np.random.seed(60)

# Pierwszy zbiór wygenerujemy, losując punkty z rozkładu jednostajnego
train_set_1d = np.random.uniform(-5, 5, size=(15, 1))

# Drugi zbiór wygenerujemy, losując punkty z rozkładu normalnego
train_set_2d = np.random.normal(-1, 3, size=(13, 2))

# Trzeci zbiór wygenerujemy, losując punkty z rozkładu wykładniczego
train_set_10d = np.random.exponential(2, size=(31, 10))
print(train_set_10d[:5, 0])

train_sets = [train_set_1d, train_set_2d, train_set_10d]

print("Parę punktów z datasetu jednowymiarowego:\n", train_set_1d[:5])
print("Parę punktów z datasetu dwuwymiarowego:\n", train_set_2d[:5])
print("Parę punktów z datasetu dziesięciowymiarowego:\n", train_set_10d[:3])

shapes = [dataset.shape for dataset in train_sets]
print("Rozmiary naszych datasetów:", *shapes)
# Pierwszy wymiar reprezentuje liczbę przykładów w datasecie
# Drugi wymiar to wymiar pojedynczego przykładu z datasetu

In [26]:
# Sprawdźmy czy datasety wylosowały się z poprawnego seedu.
similar_vals_1d = np.all(np.isclose(
    train_set_1d[:5, 0],
    np.array([-1.9912666995338126, -3.13054183658122, -1.7681732303178057, 1.6574957028830903, 0.6697080096921333])
))

similar_vals_2d = np.all(np.isclose(
    train_set_2d[:5, 0],
    np.array([-6.53671154, -0.96583933, 3.14309184, -0.54717522, -2.49814889])
))

similar_vals_10d = np.all(np.isclose(
    train_set_10d[:5, 0],
    np.array([1.92650794, 2.29209159, 0.61110025, 3.247719, 2.8237559])
))
assert similar_vals_1d and similar_vals_2d and similar_vals_10d, \
    "Wylosowane wartości są złe! Zgłoś od razu problem prowadzącemu!"

In [29]:
from numpy import linalg as LA

def mean_error(X, v):
    return LA.norm((X-v), ord=2, axis=1).mean()

def mean_squared_error(X, v):
    norms = LA.norm(X-v, ord=2, axis=1)
    return (norms**2).mean()

def max_error(X, v):
    return max(LA.norm(X-v, ord=2, axis=1))

checker.check_1_1(mean_error, mean_squared_error, max_error, train_sets)

In [42]:
def minimize_mse(X):
    return np.sum(X)/X.size

def minimize_me(X):
    return np.median(X)

def minimize_max(X):
    return (np.amax(X)+np.amin(X))/2

checker.check_1_2(minimize_me, minimize_mse, minimize_max, train_set_1d)

In [93]:
def me_grad(X, v):
    N = X.shape[0]
    temp = X - v
    for i in range(N):
        temp[i] = (-1/N)*temp[i]/LA.norm(temp[i], ord=2)
    grad = np.sum(temp, axis=0)
    return grad

def mse_grad(X, v):
    return 0

def max_grad(X, v):
    return 0

checker.check_1_3(me_grad, mse_grad, max_grad, train_sets)

ValueError: operands could not be broadcast together with shapes (0,) (2,) 